### Data processing for network creation

Imports...

In [1]:
import os
from sys import getsizeof
import pandas as pd
import numpy as np
import json
import pickle
from typing import Tuple, Dict

Some parameters...

In [2]:
# set this to True if you want to reload from CSV, which takes longer. Otherwise data is loaded from pickled files.
from_csv = False

Define loading behaviour ...

In [3]:
# define the required cbg_group_regex
cbg_group_identifier = '09009'

# define required columns and names for the two census files
cbg_b01 = {
    'file': 'cbg_b01.csv',
    'cols': [0, 159, 160],
    'names': ['cbg', 'B01003e1', 'B01003m1'],
    'dtypes': {0: 'string', 159: np.int32, 160: np.int32}
}

cbg_b25 = {
    'file': 'cbg_b25.csv',
    'cols': [0, 187, 188],
    'names': ['cbg', 'B25010e1', 'B25010m1'],
    'dtypes': {0: 'string', 187: np.float32, 188: np.float32}
}

patterns_feb = {
    'file': 'feb2020_core_poi-patterns.csv',
    'cols': [0, 25, 35],
    'names': ['placekey', 'visitor_cbg', 'cbg'],
    'dtypes': {0: 'string', 35: 'string', 25: 'string'},
}

patterns_apr = {
    'file': 'apr2020_core_poi-patterns.csv',
    'cols': [0, 25, 35],
    'names': ['placekey', 'visitor_cbg', 'cbg'],
    'dtypes': {0: 'string', 35: 'string', 25: 'string'},
}

def read(data: dict) -> pd.DataFrame:
    """
    Read raw data from a csv file.
    :param data: contains info on the data to extract.
    :returns: data in a pandas data frame.
    """
    iter_csv = pd.read_csv(f"data/{data['file']}", usecols=data['cols'], dtype=data['dtypes'], 
                           header=0, names=data['names'], iterator=True, chunksize=1000)
    
    df = pd.concat([chunk[chunk['cbg'].apply(lambda x: x.startswith(cbg_group_identifier) if not pd.isnull(x) else False)] 
                    for chunk in iter_csv])
    
    return df


Load data ...

In [4]:
%%time

if from_csv:
    
    # load from csv
    df_pat_feb = read(patterns_feb)
    df_pat_apr = read(patterns_apr)
    df_b01 = read(cbg_b01)
    df_b25 = read(cbg_b25)
    
    # save to pickle
    df_pat_feb.to_pickle('data/df_pat_feb.pkl')
    df_pat_apr.to_pickle('data/df_pat_apr.pkl')
    df_b01.to_pickle('data/df_b01.pkl')
    df_b25.to_pickle('data/df_b25.pkl')
    
else:
    
    # load from pickle
    df_b01 = pd.read_pickle('data/df_b01.pkl')
    df_b25 = pd.read_pickle('data/df_b25.pkl')
    df_pat_feb = pd.read_pickle('data/df_pat_feb.pkl')
    df_pat_apr = pd.read_pickle('data/df_pat_apr.pkl')

CPU times: user 23 ms, sys: 31.6 ms, total: 54.6 ms
Wall time: 54.1 ms


Transform the CBG files...

In [5]:
# rename columns
b01_col_names = {'B01003e1': 'population', 'B01003m1': 'population_me'}
b25_col_names = {'B25010e1': 'household_size', 'B25010m1': 'household_size_me'}

df_b01.rename(columns=b01_col_names, inplace=True)
df_b25.rename(columns=b25_col_names, inplace=True)

# convert margin of error to standard error
z_score = 1.95996

df_b01['population_se'] = df_b01['population_me'] / z_score
df_b25['household_size_se'] = df_b25['household_size_me'] / z_score

# merge data frames on census block
df_merged = pd.merge(df_b01, df_b25, on='cbg', how='outer')

In [6]:
# sanity checks with data from https://censusreporter.org/profiles/05000US09009-new-haven-county-ct/

# population should be ~850,000 
print('Population:', df_merged.sum()['population'])

# mean household should be ~2.5
print('Household size:', df_merged.mean()['household_size'])

Population: 857513.0
Household size: 2.5217413902282715


In [7]:
# check the summary stats for plausibility
df_merged.describe()

,population,population_me,population_se,household_size,household_size_me,household_size_se
count,628.000000,628.000000,628.000000,626.000000,626.000000,626.000000
mean,1365.466561,342.968153,174.987323,2.521741,0.448115,0.228635
std,652.336689,137.227720,70.015572,0.476634,0.192414,0.098173
min,0.000000,12.000000,6.122574,1.210000,0.090000,0.045919
25%,886.000000,249.500000,127.298516,2.190000,0.320000,0.163269
50%,1253.000000,325.000000,165.819711,2.520000,0.410000,0.209188
75%,1716.250000,405.250000,206.764424,2.860000,0.540000,0.275516
max,4063.000000,1047.000000,534.194575,4.000000,1.370000,0.698994


In [8]:
# check for NA values
df_merged.iloc[[x > 0 for x in df_merged.isna().sum(axis=1)], :]

,cbg,population,population_me,population_se,household_size,household_size_me,household_size_se
622,090093614022,1476,218,111.226760,NaN,NaN,NaN
627,090099900000,0,12,6.122574,NaN,NaN,NaN


In [9]:
# since we only have two rows with NA, we can impute them with the mean
df_merged.fillna(df_merged.mean(), inplace=True)

Save merged file...

In [10]:
demographics = df_merged.set_index('cbg').to_dict('index')
pickle.dump(demographics, open('data/demographics.pkl', 'wb'))

Transform pattern files ...

In [11]:
# convert JSON data to python dict
df_pat_feb.visitor_cbg = df_pat_feb.visitor_cbg.apply(lambda x: json.loads(x))
df_pat_apr.visitor_cbg = df_pat_apr.visitor_cbg.apply(lambda x: json.loads(x))

# calculate total visitors
df_pat_feb['total_visitors'] = df_pat_feb.visitor_cbg.apply(lambda x: sum(x.values()))
df_pat_apr['total_visitors'] = df_pat_apr.visitor_cbg.apply(lambda x: sum(x.values()))

Create hashmap of total `CBG-CBG` connections...

In [12]:
def create_count_hashmaps(df: pd.DataFrame, all_cbgs: set) -> Tuple[Dict, Dict]:
    """
    Create two hashmaps:
    - comb_counts: total counts of visits between two CBGs
    - trip_counts: total counts of all trips taken from each CBG
    :param df: pattern data frame
    :param all_cbgs: set containing all CBGs
    :returns: Tuple with two hashmaps
    """
    
    comb_counts = {}
    trip_counts = {}
    for i, row in df.iterrows():

        # CBG of the POI
        poi_cbg = row['cbg']

        for cbg, count in row['visitor_cbg'].items():
            
            # ignore visits from other counties
            if cbg not in all_cbgs or poi_cbg not in all_cbgs:
                continue
            
            # Combination from visitor CBG to POI CBG
            cbg_comb = (cbg, poi_cbg)
            
            # add count to combination counts
            if cbg_comb in comb_counts:
                comb_counts[cbg_comb] += count
            else:
                comb_counts[cbg_comb] = count
            
            # add count to trip counts
            if cbg in trip_counts:
                trip_counts[cbg] += count

            else:
                trip_counts[cbg] = count
                
    return comb_counts, trip_counts

def print_visit_count_info(title: str, visit_counts: dict, all_cbgs: list):
    """
    Print info about the visit counts provided.
    :param title: title to print out.
    :param visit_counts: hashmap containing the count data.
    :param all_cbgs: list of all cbgs.
    """
    print(f'{title}:')
    print(f"{len(visit_counts)} out of a possible {len(all_cbgs)**2} edges.")
    print(f"{sum(visit_counts.values())} unique visits.\n")

In [13]:
%%time
comb_counts_feb, trip_counts_feb = create_count_hashmaps(df_pat_feb, set(df_merged.cbg.tolist()))
comb_counts_apr, trip_counts_apr = create_count_hashmaps(df_pat_apr, set(df_merged.cbg.tolist()))

print_visit_count_info("Febuary", comb_counts_feb, df_merged.cbg.tolist())
print_visit_count_info("April", comb_counts_apr, df_merged.cbg.tolist())

Febuary:
52336 out of a possible 394384 edges.
777780 unique visits.

April:
29715 out of a possible 394384 edges.
327589 unique visits.

CPU times: user 3.18 s, sys: 0 ns, total: 3.18 s
Wall time: 3.18 s


Save visit counts to file...

In [14]:
pickle.dump(comb_counts_feb, open('data/comb_counts_feb.pkl', 'wb'))
pickle.dump(comb_counts_apr, open('data/comb_counts_apr.pkl', 'wb'))

pickle.dump(trip_counts_feb, open('data/trip_counts_feb.pkl', 'wb'))
pickle.dump(trip_counts_apr, open('data/trip_counts_apr.pkl', 'wb'))

# read like this:
# comb_counts_feb = pickle.load(open('data/comb_counts_feb.pkl', 'rb'))
# comb_counts_apr = pickle.load(open('data/comb_counts_apr.pkl', 'rb'))